<!DOCTYPE html>
<html>
<head>
<style>
* {
  box-sizing: border-box;
}

.column {
  float: left;
  width: 50%;
  padding: 1px;
}

/* Clearfix (clear floats) */
.row::after {
  content: "";
  clear: both;
  display: table;
}


</style>
</head>
<body>

<h2>CNN and Application to lung cancer</h2>
<h6 style="color:Tomato;">https://doi.org/10.3390/cancers14225569</h6>

<div class="row">
  <div class="column">
    <img src="cnn.png" alt="General CNN" width=500 height=100>
  </div>
  <div class="column">
    <img src="Celebr.png" alt="CNN celebs, from google images"  width=500 height=100>
  </div>
</div>

</body>
</html>



<h3 style="color: red;">Student: Rexiati Dilimulati </h3>
<h4 style="color: red;">rishat.dilmurat@gmail.com</h4>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from IPython.display import Image, HTML, display
import time
import cv2
from fastai.vision.all import *
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
torch.manual_seed(0);

In [4]:
import os
dataset_path = "/kaggle/input/lung_colon_image_set/lung_image_sets"

print(os.listdir(dataset_path))

['lung_aca', 'lung_scc', 'lung_n']


<header class="centered-header">
    <h1 style="color:Tomato;">My Mission</h1>
    <h2>Classify Lung Cancer Dataset Using CNN</h2>
</header>
<table><tr><td><img src='typeLungCanser.jpg'></td>

<img src="outputSize.jpg" alt="CNN celebs, from google images"  width=500 height=100>

<h1 style="color:Tomato;">Inception v3</h1>
<p>https://www.mdpi.com/2073-8994/14/12/2679</p>
<table><tr><td><img src='inception.jpg' style="width:600px;height:405px;"></td><td><img src='inception_01.jpg' ></td></tr>
<td><img src='inception_02.jpg' ></td></tr></table>

In [5]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        # Define branches with different pooling strategies
        self.branch1x1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 48, kernel_size=1),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True),
            nn.Conv2d(48, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.branch_pool = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),  # Using AvgPool2d instead of MaxPool2d
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        print(x.shape)
        branch1x1 = self.branch1x1(x)
        print(branch1x1.shape)
        branch5x5 = self.branch5x5(x)
        print(branch5x5.shape)
        branch_pool = self.branch_pool(x)
        print(branch_pool.shape)

        outputs = [branch1x1, branch5x5, branch_pool]
        return torch.cat(outputs, 1)


In [6]:
class InceptionB(nn.Module):
    def __init__(self, in_channels):
        super(InceptionB, self).__init__()
        # Branches
        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 384, kernel_size=3, stride=2),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True)
        )
        self.branch3x3dbl = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 128, kernel_size=3, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

    def forward(self, x):
        branch3x3 = self.branch3x3(x)
        branch3x3dbl = self.branch3x3dbl(x)
        branch_pool = self.branch_pool(x)

        outputs = [branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [7]:
class InceptionC(nn.Module):
    def __init__(self, in_channels):
        super(InceptionC, self).__init__()
        # Branches
        self.branch1x1 = nn.Sequential(
            nn.Conv2d(in_channels, 256, kernel_size=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.branch7x7x3 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=(1, 7), padding=(0, 3)),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=(7, 1), padding=(3, 0)),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.branch7x7x3dbl = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.branch_pool = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 128, kernel_size=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        branch7x7x3 = self.branch7x7x3(x)
        branch7x7x3dbl = self.branch7x7x3dbl(x)
        branch_pool = self.branch_pool(x)

        outputs = [branch1x1, branch7x7x3, branch7x7x3dbl, branch_pool]
        return torch.cat(outputs, 1)


<h1 style="color:Tomato;">Train the model</h1>

In [8]:
def train_model(model, criterion, train_loader, validation_loader, optimizer, n_epochs=4):

    N_train = len(train_loader.dataset)
    N_test = len(validation_loader.dataset)
    accuracy_list_traing = []; loss_list_training = []
    accuracy_list_test = []; loss_list_test = []
    
    for epoch in range(n_epochs):
        model.train()
        cost = 0; correct=0
        for x, y in train_loader:
            x_train = x.as_subclass(torch.Tensor)
            y_train = y.as_subclass(torch.Tensor)
            x_train, y_train = x_train.to(device), y_train.to(device)  # with GPU

            # forward + backward + optimize
            #model.to(device)
            optimizer.zero_grad()
            z = model(x_train)
            z_logits = z.logits if isinstance(z, models.InceptionOutputs) else z
            loss = criterion(z_logits, y_train)
            loss.backward()
            optimizer.step()
            
            cost = cost+loss.data
            _, yhat = torch.max(z_logits.data, 1)
            correct += (yhat == y_train).sum().item()
        accuracy = correct / N_train
        accuracy_list_traing.append(accuracy)
        loss_list_training.append(cost.item())
        
        print(f"Epoch_{epoch}:training_accuracy {accuracy:.2f} | training_loss {cost:.2f}")
        
        #perform a prediction on the validation  data 
        model.eval()
        cost = 0; correct=0
        for x_test, y_test in validation_loader:
            x_test = x_test.as_subclass(torch.Tensor)
            y_test = y_test.as_subclass(torch.Tensor)
            x_test, y_test = x_test.to(device), y_test.to(device)  # with GPU      
            
            optimizer.zero_grad()
            z = model(x_test)
            z_logits = z.logits if isinstance(z, models.InceptionOutputs) else z
            loss = criterion(z_logits, y_test)


            cost = cost+loss.data
            _, yhat = torch.max(z_logits.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct / N_test
        accuracy_list_test.append(accuracy)
        loss_list_test.append(cost.item())

        print(f"Epoch_{epoch}:testing_accuracy {accuracy:.2f} | testing_loss {cost:.2f}")
        
     
    return np.round(np.array([accuracy_list_traing, loss_list_training, accuracy_list_test, loss_list_test], dtype=float), decimals=3)

<h1 style="color:Tomato;">pytorch like loading images</h1>

In [9]:
# Create the training dataset

# Define a transform (e.g., convert to tensor, normalize)
#transform = transforms.Compose([transforms.Resize((128, 128)),
#                                transforms.ToTensor()])

# Load the dataset
#dataset = dsets.ImageFolder(root='./lung_colon_image_set/lung_image_sets/', transform=transform)

# Define the DataLoader
#dataloader = DataLoader(dataset, batch_size=50, shuffle=True)

# Iterate over the DataLoader
#for inputs, labels in dataloader:
#    print(inputs.shape, labels.shape)
    # Your training code here


<h1 style="color:Tomato;">Fastai like loading and showing images</h1>

In [10]:
# Path to the directory containing images
#dataset_path="../lung_colon_image_set/lung_image_sets/"
path = Path(dataset_path)

# Create an ImageDataLoaders object
# You can use ImageDataLoaders.from_folder, from_csv, etc., depending on your dataset structure
resize = 299
dls_lungimagesets = ImageDataLoaders.from_folder(
    path, 
    valid_pct=0.2,       # 20% of data will be used for validation
    seed=0,             # Seed for reproducibility
    item_tfms=Resize(resize),# Resize images to 224x224
    bs=50
)

# Get a batch of data
batch = dls_lungimagesets.one_batch()

In [11]:
# Access the training and validation DataLoaders
train_dls_lungimagesets = dls_lungimagesets.train
valid_dls_lungimagesets = dls_lungimagesets.valid

In [12]:
pwd

'/kaggle/working'

<h1 style="color:Tomato;"> With Inception_V3</h1>

In [13]:
# Create the criterion function
criterion = nn.CrossEntropyLoss()



# Load the pre-trained Inception V3 model
model = models.inception_v3(weights=Inception_V3_Weights.DEFAULT)

num_classes = 3
model.fc = torch.nn.Linear(model.fc.in_features, num_classes).to(device)
# the inception_v3 expects tensors with a size of N x 3 x 299 x 299

model.to(device)



# define optimizer
learning_rate = 0.01; momentum=0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)


start_time = time.time()
training_results_Inception = train_model(model, criterion, train_dls_lungimagesets, valid_dls_lungimagesets, optimizer, n_epochs=30)
end_time = time.time()

print(f'Total time: {(end_time - start_time)/60:.3f} mins')

#%timeit t_cpu @ t_cpu
#%timeit t_gpu @ t_gpu

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 144MB/s] 


Epoch_0:training_accuracy 0.96 | training_loss 22.16
Epoch_0:testing_accuracy 1.00 | testing_loss 0.59
Epoch_1:training_accuracy 0.99 | training_loss 4.44
Epoch_1:testing_accuracy 1.00 | testing_loss 0.39
Epoch_2:training_accuracy 1.00 | training_loss 1.13
Epoch_2:testing_accuracy 1.00 | testing_loss 0.03
Epoch_3:training_accuracy 1.00 | training_loss 0.23
Epoch_3:testing_accuracy 1.00 | testing_loss 0.01
Epoch_4:training_accuracy 1.00 | training_loss 0.27
Epoch_4:testing_accuracy 1.00 | testing_loss 0.03
Epoch_5:training_accuracy 1.00 | training_loss 0.12
Epoch_5:testing_accuracy 1.00 | testing_loss 0.01
Epoch_6:training_accuracy 1.00 | training_loss 0.27
Epoch_6:testing_accuracy 1.00 | testing_loss 0.01
Epoch_7:training_accuracy 1.00 | training_loss 0.06
Epoch_7:testing_accuracy 1.00 | testing_loss 0.00
Epoch_8:training_accuracy 1.00 | training_loss 0.02
Epoch_8:testing_accuracy 1.00 | testing_loss 0.00
Epoch_9:training_accuracy 1.00 | training_loss 0.49
Epoch_9:testing_accuracy 1.00

<h1 style="color:Tomato;">Show results</h1>

In [14]:
def show_data(data_sample):
    plt.imshow(data_sample[0][0].numpy(), cmap='gray')
    plt.title('y = '+ str(data_sample[1]))

In [ ]:
### """
# Plot the loss and accuracy for CNN

fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(torch.tensor(training_results_CNN[1]).numpy(), color=color)
ax1.set_xlabel('epoch', color=color)
ax1.set_ylabel('Cost', color=color)
ax1.tick_params(axis='y', color=color)
plt.title("CNN")
    
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('accuracy', color=color) 
ax2.set_xlabel('epoch', color=color)
ax2.plot(training_results_CNN[0], color=color)
ax2.tick_params(axis='y', color=color)
fig.tight_layout()
plt.savefig('LungCancer_CNN_450_450')
"""

![](LungCancer_CNN_450_450.png)